In [7]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np

In [3]:
# distribution_centers = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/최종 프로젝트/루커스 데이터셋/distribution_centers.csv')
# event = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/최종 프로젝트/루커스 데이터셋/events.csv')
# inventory_items = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/최종 프로젝트/루커스 데이터셋/inventory_items.csv')
order_items = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/최종 프로젝트/루커스 데이터셋/order_items.csv')
# orders = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/최종 프로젝트/루커스 데이터셋/orders.csv')
# products = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/최종 프로젝트/루커스 데이터셋/products.csv')
# users = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/최종 프로젝트/루커스 데이터셋/users.csv')

# RFM-D(Diversity) 데이터 전처리

In [4]:
#필요한 데이터 전처리
order_items_extracted = order_items.query('status != "Returned" and status != "Cancelled"')
order_items_extracted = order_items_extracted[['user_id', 'order_id', 'created_at', 'status', 'product_id', 'sale_price']].sort_values(by = 'user_id')

# 년-월-일 형식의 문자열로 변환
order_items_extracted['created_at'] = pd.to_datetime(order_items_extracted['created_at'], format='ISO8601')
order_items_extracted['created_at'] = pd.to_datetime(order_items_extracted['created_at'].dt.strftime('%Y-%m-%d'))
order_items_extracted = order_items_extracted.rename(columns = {'created_at' : 'date'})

In [5]:
#recency
recency_data = order_items_extracted.groupby('user_id')['date'].max().reset_index()
recency_data['Recency'] = (order_items_extracted['date'].max()-recency_data['date']).dt.days
recency_data

#frequency
frequency_data = order_items_extracted.groupby('user_id')['order_id'].nunique().reset_index()
frequency_data.rename(columns={'order_id': 'Frequency'}, inplace=True)

#monetary
monetary_data = order_items_extracted.groupby('user_id')['sale_price'].sum().reset_index()
monetary_data.rename(columns={'sale_price': 'Monetary'}, inplace=True)

#diversity
diversity_data = order_items_extracted.groupby('user_id')['product_id'].nunique().reset_index()
diversity_data.rename(columns={'product_id': 'Diversity'}, inplace=True)

rfm_data = recency_data.merge(frequency_data, on='user_id').merge(monetary_data, on='user_id').merge(diversity_data, on='user_id')


rfm_data[['Recency', 'Frequency', 'Monetary', 'Diversity']].corr()

,Recency,Frequency,Monetary,Diversity
Recency,1.000000,-0.215934,-0.107465,-0.157274
Frequency,-0.215934,1.000000,0.484705,0.735366
Monetary,-0.107465,0.484705,1.000000,0.658875
Diversity,-0.157274,0.735366,0.658875,1.000000


In [8]:
rfm_data['Recency_log'] = np.log1p(rfm_data['Recency'])
rfm_data['Frequency_log'] = np.log1p(rfm_data['Frequency'])
rfm_data['Monetary_log'] = np.log1p(rfm_data['Monetary'])
rfm_data['Diversity_log'] = np.log1p(rfm_data['Diversity'])

X_features = rfm_data[['Recency_log', 'Frequency_log', 'Monetary_log', 'Diversity_log']]
sc = StandardScaler()
X_features_sc = sc.fit_transform(X_features)